In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten() # 28*28 = 784
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(784, 512), # 784*512
            nn.ReLU(),
            nn.Linear(512, 512), # 512*512
            nn.ReLU(),
            nn.Linear(512, 10) # 512*10
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
loss_fn = nn.CrossEntropyLoss()

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0,0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10

for i in range(epochs):
    print(f"Epoch {i+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301931 [    0/60000]
loss: 2.292635 [ 6400/60000]
loss: 2.277722 [12800/60000]
loss: 2.272102 [19200/60000]
loss: 2.258940 [25600/60000]
loss: 2.222732 [32000/60000]
loss: 2.236552 [38400/60000]
loss: 2.192803 [44800/60000]
loss: 2.193199 [51200/60000]
loss: 2.171128 [57600/60000]
Test Error: 
 Accuracy: 35.8%, Avg loss: 2.163559 

Epoch 2
-------------------------------
loss: 2.173594 [    0/60000]
loss: 2.163070 [ 6400/60000]
loss: 2.112683 [12800/60000]
loss: 2.119544 [19200/60000]
loss: 2.070926 [25600/60000]
loss: 2.018736 [32000/60000]
loss: 2.044808 [38400/60000]
loss: 1.965170 [44800/60000]
loss: 1.965238 [51200/60000]
loss: 1.903326 [57600/60000]
Test Error: 
 Accuracy: 50.4%, Avg loss: 1.900500 

Epoch 3
-------------------------------
loss: 1.937493 [    0/60000]
loss: 1.908798 [ 6400/60000]
loss: 1.800939 [12800/60000]
loss: 1.821744 [19200/60000]
loss: 1.721929 [25600/60000]
loss: 1.685026 [32000/60000]
loss: 1.701589 [38400/